#### Autori: Domenico Lembo, Giuseppe Santucci and Marco Schaerf

[Dipartimento di Ingegneria informatica, automatica e gestionale](https://www.diag.uniroma1.it)

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.eu.png"
     alt="License"
     style="float: left;"
     height="40" width="100" />
This notebook is distributed with license Creative Commons *CC BY-NC-SA*

# Espressioni Regolari
1. Espressioni regolari in Python
2. Differenze tra teoria e uso in Python
3. Esempi di uso delle espressioni regolari
4. Comportamento search e finditer
5. Caratteri denotanti gruppi
6. Ricerca di stringhe con ripetizioni
7. Funzione per la sostituzione
2. Esercizio: Trovare tutte le righe di un file che, dopo averle pulite con lo strip(), iniziano e finiscono per lo stesso carattere
3. Esercizio: Calcolare quante volte in un file compare una sequenza di 3 parole consecutive con la seguente proprietà: la lettera finale della prima parola è uguale a quella iniziale della seconda e quella finale della seconda è uguale a quella iniziale della terza.
4. Esercizio: Calcolare quante volte in un file compare una sequenza di 2 parole consecutive con la seguente proprietà: Almeno 2 lettere della prima parola sono presenti anche nella seconda e nello stesso ordine, cioè se la prima lettera viene prima della seconda nella prima parola, deve venire prima anche nella seconda.
5. Esercizio: cambiare al formato americano ore e date
6. Esercizio: cambiare al formato americano ore e date con il nome del mese e numero preciso

### Espressioni regolari in Python
Python ha un modulo specifico per le espressioni regolari, chiamato `re`. Per maggiori dettagli sulle espressioni regolari in Python rimandiamo alla documentazione ufficiale [https://docs.python.org/3/library/re.html](https://docs.python.org/3/library/re.html). Le note di seguito utilizzano materiale preso dai seguenti siti:
- [https://www.python.it/doc/howto/Regex/regex-it/regex-it.html](https://www.python.it/doc/howto/Regex/regex-it/regex-it.html)
- [https://docs.python.it/html/lib/re-syntax.html](https://docs.python.it/html/lib/re-syntax.html)

### Differenze fondamentali tra la teoria delle espressioni regolari ed il loro uso in Python
Nella teoria, le espressioni regolari sono un modo per definire insiemi di stringhe in base alle loro proprietà, mentre in Python (come anche in tanti altri programmi che le usano) le espressioni regolari sono più frequentemente usate per cercare all'interno di un testo **t** (stringa) una sottostringa **s** (definita con un'espressione regolare che in Python viene spesso chiamata *pattern*) che abbia particolari proprietà. Questo utilizzo introduce alcune complicazioniche siamo costretti ad affrontare, come ad esempio:
- cosa fare quando ci sono più sottostringhe **s** che *collimano* con l'espressione regolare ? Troviamo la prima o tutte ?
- cosa fare quando ci sono 2 sottostringhe, **s1** e **s2**, che collimano con l'espressione regolare e che si sovrappongono in **t** ?
- come specificare che la sottostringa da cercare deve essere solo cercata all'inizio (oppure alla fine) della stringa ?
Vedremo che Python risolve tutti questi problemi definendo diversi metodi nella classe *re* ed un insieme più esteso di caratteri speciali di quelli visti nella parte di teoria.

#### La notazione r'stringa'
Prima di inziare, introduciamo questo modo alternativo di specificare stringhe in Python che è spesso utile nel caso delle espressioni regolari. Se in Python una stringa `s` ha davanti il carattere 'r', allora vuol dire che la stringa è *raw* (cruda, cioè da non interpretare). In questo caso TUTTI i caratteri di `s` rappresentano esattamente se stessi e NON sono caratteri di controllo tipo '\n' o '\t'. Questa notazione è molto utile nelle espressioni regolari che usano frequentemente i caratteri che nelle stringhe Python verrebbero interpretati come caratteri di controllo, come '\\'. Ad esempio:

In [ ]:
s = r'\nprova\n'
s1 = '\nprova\n'
print('La stringa s è lunga:',len(s),'e vale:\n',s)
print('La stringa s1 è lunga:',len(s1),'e vale:\n',s1)


**Nota:** Si noti che una stringa raw è una stringa legale Python preceduta da una 'r' (come visto negli esempi precedenti). La sequenza `r"\"` NON è una stringa raw, perchè `"\"` NON è una stringa legale in Python. Lo stesso dicasi per tutte le stringhe che terminano con un numero dispari di `\`. 

### Esempi di uso delle espressioni regolari
Cominciamo con dei semplici esempi di come si possano usare le espressioni regolari in Python. Successivamente vedremo più precisamente le principali operazioni ed i caratteri speciali definiti nel modulo *re*.

#### Cercare una stringa in un semplice elenco telefonico

cerchiamo "Mario" in un elenco strutturato così:
elenco='Luca Rossi 2345234 - Mario Bianchi 82342342 - Marco Verdi 342342342'

Per ora verifichiamo solo se "Mario" è presente in elenco. Per fare questo useremo la funzione `re.search(..)`.
Questa funzione ha 2 parametri, il primo è il *pattern* (di regola specificato con un'espressione regolare) ed il secondo è il testo (stringa) in cui cercare. Notate che la funzione `re.search(..)` restituisce un oggetto di tipo `match`.

In [ ]:
import re

elenco='Luca Rossi 2345234 - Mario Bianchi 82342342 - Marco Verdi 342342342'
pattern = 'Mario'

m = re.search(pattern, elenco)
if m:
    print("trovato")
else:
    print("non trovato")

#### Stampare il pezzo di stringa trovata
Il codice precedente stampava solo un messaggio, con il metodo `group()` applicato al risultato della funzione `re.search` (che è un oggetto di type match) possiamo stampare la sottostringa che ha *collimato* con l'espressione regolare (pattern)

In [ ]:
m = re.search(pattern, elenco)
if m:
    print("trovato: " + m.group())
else:
    print("non trovato")

#### Individuare il numero di telefono
Per individuare il numero di telefono dobbiamo scrivere un'espressione regolare per cercare una stringa che non contenga solo il nome ma anche il numero (che non conosciamo, ma sappiamo come è fatto). Cerchiamo quindi una sottostringa che abbia il formato:
- Mario seguito da un qualunque numero di caratteri alfabetici, spazi o cifre, basta che non ci sia il carattere '-' (che viene usato da separatore nel nostro caso)

In [ ]:
pattern = 'Mario[a-zA-Z0-9 ]*'
m = re.search(pattern, elenco)
if m:
    print("trovato: " + m.group())
else:
    print("non trovato")

#### Definizione per negazione
A volte è più semplice definire un'espressione regolare specificando cosa **NON** deve contenere invece di cosa deve contenere. Nel caso precedente, avremmo potuto definire la parte che segue 'Mario' come una sequenza di caratteri **diversi dal carattere '-'**. Il simbolo speciale per denotare la negazione è '^' (attenzione che questo stesso simbolo ha anche un altro significato, come vedremo nel seguito), il pattern completo diventa quindi 'Mario[^-]*'

In [ ]:
pattern = 'Mario[^-]*'

m = re.search(pattern, elenco)
if m:
    print("trovato: " + m.group())
else:
    print("non trovato")

#### Individuare le componenti
Molto spesso la sottostringa che cerchiamo è composta da varie parti logicamente divise. Nel nostro esempio, in realtà stiamo cercando una sottostringa composta da 3 parti distinte:
- Il *nome*, che nel nostro caso deve essere obbligatoriamente 'Mario'
- il *cognome*, che non conosciamo, ma assumiamo sia composta da sole lettere maiuscole o minuscole
- il *numero di telefono*, che assumiamo composto da sole cifre.

Queste 3 parti distinte sono separate da uno spazio bianco. Per identificare differenti parti di un'espressione regolare in Python si usano le parentesi tonde. Ogni volta che un parte dell'espressione regolare è racchiusa tra parentesi tonde, Python salva la sottostringa che collima con quella parte e la salva in un gruppo (`group()`). I gruppi vengono numerati da 1 in poi. Il gruppo `0` è un gruppo speciale e contiene l'*intera sottostringa* che ha collimato con tutta l'espressione regolare. Vediamo un esempio:

In [ ]:
pattern = '(Mario) ([A-Za-z]*) ([0-9]*)'
m = re.search(pattern, elenco)
if m:
    print("nome: " + m.group(1))
    print("cognome: " + m.group(2))
    print("telefono: " + m.group(3))
else:
    print("non trovato")

#### Suddivisione
L'espressione regolare trova una stringa composta da 3 parti separate da spazio

(Mario)| (\[A-Za-z\]\*)| (\[0-9\]\*)
:-----:|:---------:|:------:
m.group(1)| m.group(2)|m.group(3)
              
invece `m.group()` è tutta la sottostringa, uguale a `m.group(0)`.

Notate che gli spazi bianchi sono caratteri come gli altri, se inserite uno spazio nell'espressione regolare questo collima con uno spazio nel testo. Nel nostro caso, infatti, il pattern '(Mario) (\[A-Za-z\]\*) (\[0-9\]\*)' contiene uno spazio tra il nome ed il cognome ed uno tra il cognome ed il numero di telefono.

#### Trovare più sottostringhe diverse
In molti casi non vogliamo trovare una sola sottostringa che collima con la nostra espressione regolare, ma TUTTE le sottostringhe del testo che collimano con il nostro pattern. In questo caso si può usare la funzione *finditer* che ritorna un iteratore che si può usare in un ciclo e ad ogni passo è come se fosse stata fatta una   *search* (a partire dal carattere della stringa successivo a quello incluso nel match trovato al passo precedente - su questo punto vedi altri dettagli in seguito). Restituisce quindi una sequenza di match, dove ogni match contiene i gruppi che hanno collimato con le varie parti. Vediamo un esempio in cui cerchiamo nel nostro elenco tutte le persone il cui nome inizia per 'M':

In [ ]:
pattern = '(M[a-z]*) ([A-Za-z]*) ([0-9]*)'
i = re.finditer(pattern, elenco)
#calcola tutte le sottostringhe che collimano con il pattern.
#ogni sottostringa è salvata in un match con un elemento per ogni gruppo

for m in i: #m è il match contenente un singolo risultato
    print('nome: ' + m.group(1) + ' ' \
          'cognome: ' + m.group(2) + ' ' \
          'telefono: ' + m.group(3))

#### Riassumendo
Le notazioni e le funzioni che abbiamo visto sono:

- \[caratteri\] per un insieme di caratteri in alternativa
- \[^caratteri\] lo stesso ma al contrario
- \* sequenza (anche vuota)
- (gruppo) isola parti della stringa
- re.search per trovare la prima stringa
- re.finditer per un iteratore
- m.group per stampare tutta o parte della stringa trovata

In realtà Python permette di usare molti più caratteri speciali per definire le espressioni regolari, alcuni li menzioneremo più avanti, una lista più esaustiva la trovate in questo [Notebook](CaratteriSpeciali.ipynb).

#### Differenze importanti tra teoria e applicazione in Python
Come abbiamo visto nella parte di teoria, le espressioni regolari vengono usate per definire insiemi di stringhe. Ad esempio, l'espressione regolare **a\[a-z\]\*c** denota l'insieme contenente ogni stringa che inizia per **a**, seguita da un numero qualunque (anche 0) di altre lettere e poi terminata da una **c**. La definizione è precisa e non ambigua, ma quando questa espressione regolare viene usata per cercare una stringa di questo insieme all'interno di un testo, ci sono varie possibilità. Vediamo un esempio e come si comporta il modulo *re* di Python e le sue funzioni.

Assumiamo che il nostro pattern sia quello descritto sopra e vediamo il comportamento in 2 casi:

In [ ]:
pattern = 'a[a-z]*c'
s1 = 'palladicartasualberocaduto'
s2 = 'pallacaduta casabianca'

##### Applichiamo i metodi search e finditer
Applicando i 2 metodi nelle 2 situazioni otteniamo:

In [ ]:
print('ricerca nel testo:',s1)
m = re.search(pattern,s1)
if m:
    print(m.group())

i = re.finditer(pattern,s1)
for m in i:
    print(m.group())
    
print() #stampa riga di separazione
print('ricerca nel testo:',s2)
m = re.search(pattern,s2)
if m:
    print(m.group())

i = re.finditer(pattern,s2)
for m in i:
    print(m.group())

##### Risultato
Come si vede, nel caso di s1 sia la search che la finditer hanno trovato una sola stringa ed è la stringa **più lunga** dell'insieme delle stringhe che collimano con l'espressione regolare, anche se c'erano altre stringhe in s1 che collimavano con il pattern, come ad esempio 'alladic' o 'alberoc'. Nel secondo caso la finditer trova 2 soluzioni distinte. Siamo ora in grado di capire meglio come si comportano la search e la finditer.

### Comportamento search e finditer
Possiamo descrivere il comportamente delle funzioni search e finditer come segue:
- `re.search(pattern,testo)`: Cerca la **prima (più a sinistra)** sottostringa di testo che collima con l'espressione regolare e, se ci sono più stringhe che collimano ed iniziano nella stessa posizione, prende **la più lunga**.
- `re.finditer(pattern,testo)`: Si comporta come la search per cercare ogni soluzione e quando ne trova una riparte a cercarne altre **dalla posizione successiva a quelle occupate dalla precedente soluzione**, cioè cerca tutte soluzioni che **non abbiano alcuna sovrapposizione**.

#### Modifica comportamento search e finditer
Per specificare che si vuole invece cercare la **più corta** stringa che collima con l'espressione regolare bisogna usare degli operatori diversi, come spiegato meglio nel notebook [Caratteri Speciali](CaratteriSpeciali.ipynb) menzionato già in precedenza. A titolo di esempio, nella ricerca appena vista, dobbiamo sostituire l'operatore \* con \*?, che denota la ricerca della sottostringa **più breve** che collima con l'espressione regolare.

In [ ]:
pattern2 = 'a[a-z]*?c' #pattern modificato per indicare la ricerca della sottostringa più breve
print('ricerca nel testo:',s1)
m = re.search(pattern2,s1)
if m:
    print(m.group())

i = re.finditer(pattern2,s1)
for m in i:
    print(m.group())
    
print() #stampa riga di separazione
print('ricerca nel testo:',s2)
m = re.search(pattern2,s2)
if m:
    print(m.group())

i = re.finditer(pattern2,s2)
for m in i:
    print(m.group())

##### Nuovo risultato
Come si vede, con questa nuova versione del pattern, nel caso di s1 la search trova una stringa più corta e la finditer ora trova due soluzioni che collimano con l'espressione regolare. Nel secondo caso non ci sono differenze in quanto la stringa trovata da search era l'unica possibile.

#### Metodi fondamentali per la ricerca
Oltre i metodi `search()` e `finditer()` sono disponibili altri 2 metodi. Ricapitoliamoli brevemente tutti.

Metodi/Attributi 	Scopo 
* `match(pattern, string, flags=0)`	Determina se la RE (regular expression) *pattern* corrisponde (match) **all'inizio** della stringa *string* e restituisce un oggetto di tipo match, che, intuitivamente, è un oggetto che rappresenta la corrispondenza trovata. Restituisce `None` se nessuna corrispondenza è stata trovata. 
* `search(pattern, string, flags=0)`	Analizza la stringa *string* cercando la **prima posizione** in cui la RE *pattern* produce una corrispondenza e restituisce l'oggetto di tipo match corrispondente. Restituisce `None` se nessuna posizione nella stringa corrisponde con la RE.
* `findall(pattern, string, flags=0)`	Restituisce tutte le corrispondenze **non sovrapposte** della RE *pattern* nella stringa *string*, sotto forma di un elenco di stringhe. La stringa *string* viene scansionata da sinistra a destra e le corrispondenze vengono restituite nell'ordine trovato. 
* `finditer(pattern, string, flags=0)`	Trova tutte le corrispondenze **non sovrapposte** della RE *pattern* nella stringa *string*, e le restituisce in un iteratore che produce un oggetto di tipo match alla volta. La stringa *string* viene scansionata da sinistra a destra e gli oggetti di tipo match vengono restituiti nell'ordine trovato.

Le principali opzioni disponibili (*flags*) sono: 
* `re.DOTALL` Permette al carattere speciale **'.'** di collimare con ogni carattere, includendo il fine riga ('\n'), altrimenti non considerato;
* `re.IGNORECASE`	Cerca la corrispondenza dei caratteri senza fare distinzione tra maiuscolo e minuscolo
* `re.MULTILINE`  Tratta ogni riga come se fosse l'inizio della stringa, quindi **'^'** collima con l'inizio di ogni riga e **'\$'** collima con la fine di ogni riga.

Per specificare più di una opzione, si utilizza l'operatore | per connetterle. Per esempio

`re.search(pattern,string,flags=re.IGNORECASE|re.MULTILINE)`

`flags=` può essere omesso, ottenendo quindi

`re.search(pattern,string,re.IGNORECASE|re.MULTILINE)`

Ovviamente, se l'importazione della libreria `re` è avvenuta con il comando 

`from re import *`

è possibile anche scrivere `DOTALL` al posto di `re.DOTALL` (analogamente per le altre opzioni).

I metodi `match()` e `search()` restituiscono un oggetto di tipo `match`, mentre la funzione `finditer()` restituisce un iteratore che genera un oggetto di tipo `match` per volta. Gli oggetti di tipo `match` hanno questi metodi fondamentali:

| Metodo |	Scopo |
|:---:|:---:|
|`group()` |	Restituisce la stringa che collima con RE |
|`start()` |	Restituisce la posizione iniziale della sottostringa che collima con RE |
| `end()` |	Restituisce la posizione finale della sottostringa che collima con RE|
|`span()`	| Restituisce la tupla con posizione iniziale e finale della sottostringa che collima con RE|

### Caratteri denotanti gruppi
Alcune delle sequenze speciali che iniziano con "\\" rappresentano degli insiemi predefiniti di caratteri che sono spesso utili, come l'insieme delle cifre, l'insieme delle lettere o l'insieme di tutto ciò che non è un carattere di spaziatura. Sono disponibili le seguenti sequenze predefinite:

* **\d** Corrisponde ad ogni cifra decimale; equivale alla classe \[0-9\].
* **\D** Corrisponde ad ogni carattere che non sia una cifra; è equivalente alla classe \[^0-9\].
* **\s** Corrisponde ad ogni carattere di spaziatura; è equivalente alla classe \[ \t\n\r\f\v\] (si ricorda che \t è un tab orizzontale, \n è una andata a capo, \r è un carriage return - riporta il cursore all'inizio della riga, \f è un form feed - avanza la stampa alla pagina successiva, \v è un tab verticale).
* **\S** Corrisponde ad ogni carattere che non sia un carattere di spaziatura; è equivalente alla classe \[^\t\n\r\f\v\].
* **\w** Corrisponde ad ogni carattere alfanumerico (più l'underscore '\_'); è equivalente alla classe \[a-zA-Z0-9\_\].
* **\W** Corrisponde ad ogni carattere che non sia alfanumerico e non sia l'underscore; è equivalente alla classe \[^a-zA-Z0-9\_\].

### Ricerca di stringhe con ripetizioni

* **\\numero** Corrisponde al contenuto del gruppo con lo stesso numero. I gruppi vengono numerati a partire da 1. Per esempio, '(.+)_\\\1' è una espressione regolare che corrisponde, ad esempio, a 'the_the' o a '55_55', ma non a 'the_sun' o a 'thethe'. Notate che il carattere '\\' va ripetuto 2 volte, poiché nelle stringhe il '\\' ha un significato particolare (denota caretteri di escape). Se volete evitare la ripetizione potete usare la stringa *raw* r'(.+) \1'. In altri termini, nell'espressione precedente, (.+) corrisponde ad una sequenza di un numero arbitraio ma maggiore di 1 di caratteri qualsiasi; una volta individuato questo gruppo, \1 corrisponde ad una sua replica (che nell'espressione regolare dell'esempio deve essere separata dalla precedente da un underscore).

#### Vediamo degli esempi

In [ ]:
import re

# Cerchiamo nel file "I_Malavoglia.txt" la più lunga stringa INIZIALE 
# che inizia per una vocale e finisce per la STESSA vocale, 
# ignorando maiuscole e minuscole
testo=open("I_Malavoglia.txt",encoding="UTF-8").read()
regex1 = r'([aeiou]).*\1'
ris1 = re.match(regex1,testo,re.IGNORECASE)
print(ris1.span(),ris1.group())

# Cerchiamo nel file "I_Malavoglia.txt" la prima riga che INIZIA 
# per una vocale e finisce per la STESSA vocale seguita 
# dal punto e dal carattere di riga nuova '\n', 
# ignorando maiuscole e minuscole
regex2 = r'^([aeiou]).*\1\.\n'
ris2 = re.search(regex2,testo,re.IGNORECASE | re.MULTILINE)
print(ris2.span(),ris2.group())


Si noti che nell'ultimo esempio il primo `.` è da intendersi come il simbolo speciale che nelle espressioni regolari corrisponde a qualunque carattere, mentre il secondo `.` corrisponde effettivamente ad un punto nella stringa. Per distinguere i due casi (ed intendere il secondo punto come carattere e non come simbolo speciale) abbiamo utilizzato il `\` prima del secondo punto (avremmo fatto lo stesso per cercare nel stringa un asterisco (`*`) o un qualunque altro carattere che è utilizzato come sibolo speciale nell'espressione regolare).

In [ ]:
# Contiamo nel file "I_Malavoglia.txt" quante righe INIZIANO per una
# vocale e finiscono per la STESSA vocale seguita dal punto e dal 
# carattere di riga nuova '\n', ignorando maiuscole e minuscole
ris3 = re.findall(regex2,testo,re.IGNORECASE | re.MULTILINE)
print(len(ris3))

In [ ]:
# Cerchiamo nel file "I_Malavoglia.txt" tutte le righe che INIZIANO 
# per una vocale e finiscono per la STESSA vocale seguita dal punto e
# dal carattere di riga nuova '\n', ignorando maiuscole e minuscole
ris4 = re.finditer(regex2,testo,re.IGNORECASE | re.MULTILINE)
for m in ris4:
    print(m.span(),m.group())

### Funzione per la sostituzione
Ancora più utili possono essere le espressioni regolari quando si vogliono fare complesse sostituzioni. La funzione fondamentale per effettuare sostituzioni usando le espressioni regolari è:
* `sub(pattern, repl, string, count=0, flags=0)` Restituisce una nuova stringa dove tutte le occorrenze **non sovrapposte** dell'espressione regolare (*pattern*) nella stringa (*string*) sono sostituite dal rimpiazzo (*repl*). Il parametro *count* serve per limitare (eventualmente) il numero di sostituzioni da effettuare.

Vediamo come esempio la modifica delle istruzioni print dalla versione 2 alla 3 di Python, vedi il file [prova.py](prova.py):

In [ ]:
# Modifica le istruzioni print di Python2 nella versione corretta per Python3
filein = open('prova.py',encoding='UTF-8')
fileout = open('nuovoprova.py','w',encoding='UTF-8')
pattern = 'print (.*)\n'
repl = r'print(\1)\n'
for riga in filein:
    if 'print' in riga:
        nuova = re.sub(pattern,repl,riga)
        print(nuova)
        fileout.write(nuova)
    else:
        fileout.write(riga)
filein.close()
fileout.close()

#### Discussione
il programma di sopra funziona correttamente nella maggior parte dei casi, ma fallisce sull'ultima riga del file [prova.py](prova.py), in quanto l'ultima riga NON è terminata da un '\n'. Per rendere più completa la nostra soluzione dobbiamo prevedere l'opzionalità del '\n' finale, ma allo stesso tempo stare attenti ad ottenere un risultato in cui non viene modificata la struttura del file originario. Per fare questo introduciamo un secondo gruppo che include il solo '\n' finale (se presente) oppure la stringa vuota. Questo gruppo viene inserito alla fine della relativa istruzione di print.

In [ ]:
# Modifica le istruzioni print di Python2 nella versione corretta per Python3
filein = open('prova.py',encoding='UTF-8')
fileout = open('nuovoprova2.py','w',encoding='UTF-8')
pattern = 'print (.*)(\n?)'
repl = r'print(\1)\2'
for riga in filein:
    if 'print' in riga:
        nuova = re.sub(pattern,repl,riga)
        print(nuova)
        fileout.write(nuova)
    else:
        fileout.write(riga)
filein.close()
fileout.close()

### Esercizio:
Trovate tutte le righe di un file che, dopo averle pulite con lo `strip()`, iniziano e finiscono per lo stesso carattere (ignorando la differenza fra maiuscole e minuscole), seguito eventualmente dal punto, ed inseritele in una lista.

In [ ]:
def listaRighe(file):
    filein = open(file,encoding='UTF-8')
    pattern = '^(\w).*\\1\.?$'
    lista = []
    for riga in filein:
        rigaPulita = riga.strip()
        if re.search(pattern,rigaPulita,re.IGNORECASE):
            lista.append(rigaPulita)
    return lista

ris = listaRighe("I_Malavoglia.txt")
print(len(ris),ris[:10])

### Esercizio:
Contare quante volte in un file compare una sequenza di 3 parole consecutive con la seguente proprietà: la lettera finale della prima parola è uguale a quella iniziale della seconda e quella finale della seconda è uguale a quella iniziale della terza.
**Nota:** Si assuma che la seconda parola di ciascuna sequenza contenga almeno due lettere e non si considerino sequenze sovrapposte (ad esempio, nella stringa 'a ab ba ac c' c'è comunque una sola sequenza, perchè ba deve essere considerato parte di una sola sequenza)

In [ ]:
from tester import tester_fun
import re

def conta(file):
    testo=open(file,"r",encoding="UTF-8").read()
    regex = r'[a-z]*([a-z])[^a-z]+\1[a-z]*([a-z])[^a-z]+\2[a-z]*'
    ris = re.findall(regex,testo,re.IGNORECASE)
    return len(ris)

counter_test_positivi = 0

counter_test_positivi += tester_fun(conta,['file1.txt'],2)
counter_test_positivi += tester_fun(conta,['file2.txt'],3)
counter_test_positivi += tester_fun(conta,['file3.txt'],4)

print('La funzione',conta.__name__,'ha superato',counter_test_positivi,'test')

### Esercizio:
Calcolare quante volte in un file compare una sequenza di 2 parole consecutive con la seguente proprietà: Almeno 2 lettere della prima parola sono presenti anche nella seconda e nello stesso ordine, cioè se la prima lettera viene prima della seconda nella prima parola, deve venire prima anche nella seconda.

**Nota:** non si considerino sequenze sovrapposte (ad esempio, nella stringa 'asta sito toro' c'è comunque una sola sequenza, perchè 'sito' deve essere considerato parte di una sola sequenza)

In [ ]:
from tester import tester_fun

import re

def conta2(file):
    testo=open(file,"r",encoding="UTF-8").read()
    regex = r'[a-z]*([a-z])[a-z]*([a-z])[a-z]*[^a-z]+[a-z]*\1[a-z]*\2[a-z]*'
    ris = re.findall(regex,testo,re.IGNORECASE)
    return len(ris)

counter_test_positivi = 0

counter_test_positivi += tester_fun(conta2,['file1.txt'],1)
counter_test_positivi += tester_fun(conta2,['file2.txt'],1)
counter_test_positivi += tester_fun(conta2,['file3.txt'],2)

print('La funzione',conta2.__name__,'ha superato',counter_test_positivi,'test')

### Esercizio: cercare tutte le date presenti in un file
Vogliamo stampare tutte le date presenti in un file, assumiamo che le date siano nel formato 21/11/2013. Useremo come test il file 'testoConDate.txt'.

In [ ]:
file = open('testoDateCF.txt', encoding = 'UTF-8')
testo = file.read()

In [ ]:
# Prima versione con formato rigido
pattern1 = '[0-9][0-9]/[0-9][0-9]/[0-9][0-9][0-9][0-9]'
i = re.finditer(pattern1,testo)
for m in i:
    print(m.group())

In [ ]:
# Seconda versione giorno e mese possono avere anche una sola cifra
pattern2 = '[0-9]?[0-9]/[0-9]?[0-9]/[0-9][0-9][0-9][0-9]'
i = re.finditer(pattern2,testo)
for m in i:
    print(m.group())

In [ ]:
# Terza versione giorno e mese possono avere anche una sola cifra e l'anno può avere 2 o 4 cifre
pattern2 = '[0-9]?[0-9]/[0-9]?[0-9]/([0-9][0-9])?[0-9][0-9]'
i = re.finditer(pattern2,testo)
for m in i:
    print(m.group())

### Esercizio: cercare i codici fiscali in un file
Scrivere una funzione che riceve in ingresso il nome di un file e restituisce due liste, ordinate alfabeticamente, di tutti i codici fiscali **corretti** presenti nel file, la prima contenente i codici fiscali delle donne e la seconda degli uomini.

Il formato del codice fiscale è:
1. tre maiuscole
2. spazio (opzionale)
3. tre maiuscole
4. spazio (opzionale)
5. due cifre, una maiuscola, due cifre
6. spazio (opzionale)
7. maiuscola, tre cifre, maiuscola

Il gruppo 5 denota la data di nascita, le prime 2 cifre reppresentano il giorno, la lettera denota il mese e le ultime 2 cifre denotano l'anno di nascita. Per le donne, il giorno di nascita viene aumentato di 40. Per verificare la correttezza dovete verificare che il giorno di nascita sia un valore compreso tra 1 e 31 (per gli uomini) oppure tra 40 e 71 (per le donne) e che la lettera sia una tra le lettere 'ABCDEHLMPRST' che rappresentano i 12 mesi dell'anno.

In [ ]:
# La soluzione proposta prima estrae tutte le stringhe che hanno
# la struttura del codice fiscale, verificando poi se siano corretti
# e decidendo se si riferiscono a uomini o donne.
import re

def trovaCF(file):
    #definiamo il pattern seguendo le regole viste sopra
    #Notate le prentesi intorno alle cifre del giorno e alla cifra
    #del mese, servono per estrarre questi campi e poterli analizzare
    #per poter verificare correttezza e sesso.
    pattern='[A-Z]{3} ?[A-Z]{3} ?([0-9]{2})([A-Z])[0-9]{2} ?[A-Z][0-9]{3}[A-Z]'
    s = open(file).read()
    i = re.finditer(pattern, s)
    l_u = []
    l_d = []
    mesi = 'ABCDEHLMPRST'
    for m in i:
        cf = m.group(0)
        giorno = int(m.group(1))
        mese = m.group(2)
        if (1 <= giorno <= 31) or (41 <= giorno <= 71) and mese in mesi:
            if giorno <= 31:
                l_u.append(cf)
            else:
                l_d.append(cf)
    l_u.sort()
    l_d.sort()
    return l_u,l_d

trovaCF('testoDateCF.txt')

#### Discussione
Il programma di sopra risolve il problema, ma non è completamente preciso. Per essere precisi, bisognerebbe verificare che il numero sia coerente con il mese (Aprile, Giugno, Settembre e Novembre) hanno 30 giorni e Febbraio ne ha 28 (o 29). Nell'Esercizio 4 inlcuso nella lista degli esercizi finali vi viene chiesto di rendere il controllo più preciso sulle date.

### Esercizio: cambiare le date al formato americano
Dobbiamo scrivere una funzione che riceve una stringa contenente delle date nel formato dd/mm/yyyy (ma ammettete anche che il giorno e mese possano essere scritti con una sola cifra e l'anno con sole due cifre) e le scrive nel formato americano con solo 2 cifre per gli anni (mm/dd/yy)

In [ ]:
def converti(s):
    pattern = '(\d?\d)/(\d?\d)/(\d\d)?(\d\d)'
    repl = r'\2/\1/\4'
    return re.sub(pattern,repl,s)

print(converti('01/02/1972 1/3/89 11/12/2011'))

### Esercizio: cambiare al formato americano ore e date con il nome del mese
Vogliamo convertire tutte le date ed ore presenti in un file nel formato americano con mesi per esteso, numeri con i suffissi ed anno a 4 cifre (assumere che tutte le date si riferiscano al secolo XX e mettere quindi all'inizio 19, se necessario). Ad esempio, la data **11/07/89** deve diventare **July 11th 1989**. Per le ore, si accettano sia nel formato **14:00** che **13.20** e vengono convertite nel formato americano con i **:** come separatore. Vediamo prima la funzione ausiliaria che converte le ore:

In [ ]:
# Converte le ore in formato 24h al formato 12h + am/pm.
# Attenzione alle eccezioni per i casi della mezzanotte e mezzogiorno
# per cui le 00:20 diventano 12:20am e le 12:30 diventano 12:30pm
def oreAmericane(s):
    coppia = s.split(':')
    ore = int(coppia[0])
    minuti = coppia[1]
    if ore >= 13:
        oraAm = str(ore-12)
        ampm = 'pm'
    elif ore == 12:
        oraAm = '12'
        ampm = 'pm'
    elif ore == 0:
        oraAm = '12'
        ampm = 'am'
    else:
        oraAm = str(ore)
        ampm = 'am'
    if len(oraAm) == 1:
        oraAm = '0'+oraAm
    return oraAm+':'+minuti+ampm

print('21:10',oreAmericane('21:10'))
print('01:10',oreAmericane('01:10'))
print('00:10',oreAmericane('00:10'))
print('12:10',oreAmericane('12:10'))

#### Funzioni ausiliarie `month()` e `day()`
Scriviamo le funzioni ausiliarie che convertono il mese nel nome esteso inglese ed il giorno con il suffisso appropriato

In [ ]:
#Converte il mese numerico nel nome in inglese
def month(s):
    n = int(s)
    months = ('January','February','March','April','May','June','July','August','September','October','November','December')
    return months[n-1]

def day(s):
    n = int(s[-1])
    if len(s) > 1:
        d = s[0] #Decina del giorno, se assente vale None
    else:
        d = None
    if n == 1 and d != '1': #Fa eccezione 11
        return s+'st'
    if n == 2 and d != '1': #Fa eccezione 12
        return s+'nd'
    if n== 3 and d != '1': #Fa eccezione 13
        return s+'rd'
    return s+'th'

print('04',month('04'))
print('6',month('6'))
print('21',day('21'))
print('11',day('11'))
print('23',day('23'))
print('5',day('5'))

#### Funzione complessiva
Possiamo ora scrivere la funzione complessiva chiamata **inglesizza(s)**

In [ ]:
def inglesizza(filein,fileout):
    f1 = open(filein,encoding= 'UTF-8')
    f2 = open(fileout,'w',encoding= 'UTF-8')
    s = f1.read()
    patternDate = '(\d?\d)/(\d?\d)/(\d\d)?(\d\d)'
    patternOre = '(\d?\d)[:.](\d\d)'
    for match in re.finditer(patternDate,s):
        d = day(match.group(1))
        m = month(match.group(2))
        if match.group(3) == None:
            year = '19'+match.group(4)
        else:
            year = match.group(3)+match.group(4)
        nuova = m+' '+d+' '+year
        s = s.replace(match.group(),nuova)
    for match in re.finditer(patternOre,s):        
        nuova = oreAmericane(match.group(1)+':'+match.group(2))
        s = s.replace(match.group(),nuova)
    f2.write(s)
    f1.close()
    f2.close()

inglesizza('Esempio Date Ore.txt','risultato.txt')

### Esercizi
Completate questi esercizi prima di cominciare il prossimo argomento

#### Esercizio 1: 
Scrivere un programma che prende in ingresso il nome di un file e restituisce la lista (ordinata alfabeticamente e senza ripetizioni) delle parole che contengono al loro interno almeno 3 doppie, cioè 3 coppie di lettere uguali consecutive come ad esempio 'arrabbattarsi'. Non fate differenza tra maiuscole e minuscole.

In [ ]:
from tester import tester_fun

import re

def trovaParole(file):
    f=open(file,'r',encoding='UTF-8')
    testo=f.read().strip().split()
    cont=set()
    for parola in testo:
        regex = r'[a-z]*([a-z])\1[a-z]*([a-z])\2[a-z]*([a-z])\3[a-z]*'
        ris = re.search(regex,parola,re.IGNORECASE)
        if ris:
            cont.add(ris.group())
    f.close()
    fine=list(cont)
    fine.sort()
    return fine
    
tester_fun(trovaParole,['I_Malavoglia_50.txt'],['scappellotto'])
tester_fun(trovaParole,['I_Malavoglia.txt'],['abboccherebbero', 'ammazzeranno', 'arrabbattarsi', 'azzuffassero', 'cappellaccio', 'cappelletta', 'cappellette', 'scappellotto', 'uccellaccio'])

#### Esercizio 2:
Generalizzare l'esercizio 1 mettendo il numero di doppie come parametro, cioè scrivere una funzione che prende come parametro il nome di un file ed il numero (minimo) n di doppie che le parole devono contenere e la lista (ordinata alfabeticamente e senza ripetizioni) delle parole che contengono al loro interno almeno n doppie, cioè n coppie di lettere uguali consecutive. Non fate differenza tra maiuscole e minuscole.

In [ ]:
from tester import tester_fun

import re

def trovaParole2(file,n):
    f=open(file,'r',encoding='UTF-8')
    testo=f.read().strip().split()
    cont=set()
    regex = r'[a-z]*'
    for i in range(1,n+1):
        regex=regex+'([a-z])'+'\\'+str(i)+'[a-z]*'
    for parola in testo:
        ris = re.search(regex,parola,re.IGNORECASE)
        if ris:
            cont.add(ris.group().lower())
    f.close()
    fine=list(cont)
    fine.sort()
    return fine 
    
tester_fun(trovaParole2,['I_Malavoglia_50.txt',3],['scappellotto'])
tester_fun(trovaParole2,['I_Malavoglia_50.txt',2],['addii', 'affrettati', 'ammarrata', 'apparecchiare', 'arricchirsi', 'arricchisci', 'arricchito', 'donnicciuole', 'fazzoletto', 'gruzzoletto', 'parommella', 'parrocchia', 'ricchezze', 'scappellotto', 'soddisfatti', 'soggetto', 'villaggio', 'vorrebbe', 'zuppidda'])
tester_fun(trovaParole2,['I_Malavoglia.txt',3],['abboccherebbero', 'ammazzeranno', 'arrabbattarsi', 'azzuffassero', 'cappellaccio', 'cappelletta', 'cappellette', 'scappellotto', 'uccellaccio'])

#### Esercizio 3:
Calcolare quante volte in un file compare una sequenza di 3 parole consecutive con la seguente proprietà: la lettera finale della prima parola è uguale a quella iniziale della seconda e quella finale della seconda è uguale a quella iniziale della terza.

**Nota:** Si assuma che la seconda parola di ciascuna sequenza contenga almeno due lettere e non si considerino sequenze sovrapposte (ad esempio, nella stringa 'a ab ba ac c' c'è comunque una sola sequenza, perchè ba deve essere considerato parte di una sola sequenza)

In [ ]:
from tester import tester_fun

import re

def conta(file):
    f=open(file,'r',encoding='UTF-8')
    testo=f.read().strip()
    totale=0
    pattern=r'[a-z]*([a-z])[^a-z]+\1[a-z]*([a-z])[^a-z]+\2[a-z]*[^a-z]'
    tot=re.finditer(pattern,testo,re.IGNORECASE)
    if tot:
        for caso in tot:
            totale+=1    
    f.close()
    return totale

    
counter_test_positivi = 0

counter_test_positivi += tester_fun(conta,['file1.txt'],2)
counter_test_positivi += tester_fun(conta,['file2.txt'],3)
counter_test_positivi += tester_fun(conta,['file3.txt'],4)

print('La funzione',conta.__name__,'ha superato',counter_test_positivi,'test')


#### Esercizio 4:
Il programma che abbiamo visto per l'estrazione dei codici fiscali risolve il problema, ma non è completamente preciso. Per essere precisi, bisognerebbe verificare che il numero sia coerente con il mese (Aprile, Giugno, Settembre e Novembre) hanno 30 giorni e Febbraio ne ha 28 (o 29). Provate a completare quell'esercizio introducendo questi controlli addizionali. Assumete che un anno sia bisestile se è un multiplo di 4 (semplificando così leggermente il calcolo).

In [1]:
from tester import tester_fun

import re

def trovaCF2(file):
    f=open(file,'r',encoding='UTF-8')
    lista=[]
    giusti='ABCDEHLMPRST'
    for riga in f:
        riga1=riga.strip()
        pattern=r'[A-Z]{3}[^A-Z]*[A-Z]{3}[^A-Z]*(\d{2})([A-Z])(\d{2})[^A-Z]*[A-Z]\d{3}[A-Z]'
        codice=re.search(pattern,riga1)
        if codice:
            gg=int(codice.group(3))
            if codice.group(2) not in giusti:
                continue
            elif gg>71 or 31 < gg <41:
                continue
            else:
                for mese in range(len(giusti)):
                    if codice.group(2)==giusti[mese]:
                        if mese<10:
                            mm='0'+str(mese+1)
                        else:
                            mm=str(mese+1)
                giorno=codice.group(3)
                anno=codice.group(1)
                if int(giorno)>40:
                    giorno=str(int(giorno)-40)   
                if int(anno)<=18:
                    anno='20'+anno
                if int(codice.group(1))>18:
                    anno='19'+anno
                if int(giorno)<10:
                    giorno='0'+giorno
                if int(giorno)>28 and int(mm)in[2,4,6,9,11]:
                    if int(giorno)==31 and int(mm)in[4,6,9,11]:
                        continue
                    if int(giorno)>28 and int(mm)==2:
                        continue
                lista.append(giorno+'/'+mm+'/'+anno)
        else:
            continue
    f.close()
    print(lista)
    return len(lista)
    
counter_test_positivi = 0

counter_test_positivi += tester_fun(trovaCF2,['testoDateCF.txt'],2)
counter_test_positivi += tester_fun(trovaCF2,['testoDateCF2.txt'],3)
counter_test_positivi += tester_fun(trovaCF2,['testoDateCF3.txt'],4)

print('La funzione',trovaCF2.__name__,'ha superato',counter_test_positivi,'test')
   

Test funzione: trovaCF2 

Input funzione: 'testoDateCF.txt' 

Output atteso:
 2 

----- print interne funzione -----

['17/04/2011', '17/04/1933', '17/04/1941', '17/04/2013']
----------------------------------

Output ottenuto:
 4 

Risultato Test: NEGATIVO

******************************

Test funzione: trovaCF2 

Input funzione: 'testoDateCF2.txt' 

Output atteso:
 3 

----- print interne funzione -----

['17/04/1931', '17/04/1933', '17/04/1971']
----------------------------------

Output ottenuto:
 3 

Risultato Test: POSITIVO

******************************

Test funzione: trovaCF2 

Input funzione: 'testoDateCF3.txt' 

Output atteso:
 4 

----- print interne funzione -----

['17/04/1931', '17/04/1933', '17/04/1971']
----------------------------------

Output ottenuto:
 3 

Risultato Test: NEGATIVO

******************************

La funzione trovaCF2 ha superato 1 test
